In [156]:
from ankipandas import Collection
from jamdict import Jamdict
from jamdict.util import LookupResult
from chirptext.deko import is_kana

import pandas as pd

pd.set_option('display.max_columns', None)

In [157]:
col = Collection()

INFO: Loaded db from /home/matthieu/.local/share/Anki2/User 1/collection.anki2


In [158]:
notes = col.notes
notes = notes[notes.nmodel == 'Japanese-Core']
notes = notes.fields_as_columns()
notes.head()

,nguid,nmod,nusn,ntags,nmodel,nfld_Expression,nfld_Meaning,nfld_Reading,nfld_Grammar,nfld_Additional_meaning
nid,,,,,,,,,,
1559205847902,eT9NjCgoYk,1560104752,744,[jlpt-n5],Japanese-Core,学校,school,学校[がっこう],Noun,
1559205847903,"if1m,QMs>O",1560104752,744,"[jlpt-n3, jlpt-n5, jlpt-n3, jlpt-n5]",Japanese-Core,川,"river, stream",川[かわ],Noun,"River, the ... river"
1559205847904,o&Ra:0!aZS,1560104752,744,"[jlpt-n5, jlpt-n5]",Japanese-Core,手,"hand, arm",手[て],Noun,"forepaw, foreleg; handle; hand, worker, help; ..."
1559205847905,Q|I^ax+dT=,1560104752,744,"[jlpt-n5, jlpt-n5]",Japanese-Core,戸,door (esp. Japanese-style),戸[と],Noun,"shutter, window shutter; entrance (to a home);..."
1559205847906,G@w%QE?!%L,1560104752,744,"[jlpt-n1, jlpt-n5, jlpt-n1, jlpt-n5]",Japanese-Core,眼鏡,"glasses, eyeglasses, spectacles",眼鏡[めがね],Noun,"judgment, judgement, discrimination, discernme..."


In [159]:
jmd = Jamdict()

In [160]:
def readings(kanji: str):
    on = []
    kun = []
    mean = []

    if is_kana(kanji):
        return None

    k = jmd.lookup(kanji)

    for c in k.chars:
        for r in c.rm_groups:
            on = on + [r.value for r in filter(lambda x : x.r_type == 'ja_on', r.readings)]
            kun = kun + [r.value for r in filter(lambda x : x.r_type == 'ja_kun', r.readings)]
            mean = mean   + [r.value for r in filter(lambda x : x.m_lang == '', r.meanings)]
        
    return {'kanji' : kanji, 'on': set(on), 'kun' :set(kun), 'meaning': set(mean)}

def define(s: str):
    i = 1

    r = {}

    for e in filter(None, [readings(k) for k in s]):
        r.update({f'nfld_{k}_{i}': ', '.join(v) for k, v in e.items()})
        i = i + 1

    return r

define('学校')

{'nfld_kanji_1': '学',
 'nfld_on_1': 'ガク',
 'nfld_kun_1': 'まな.ぶ',
 'nfld_meaning_1': 'learning, science, study',
 'nfld_kanji_2': '校',
 'nfld_on_2': 'コウ, キョウ',
 'nfld_kun_2': '',
 'nfld_meaning_2': 'exam, school, proof, correction, printing'}

In [179]:
%%time

kanjis = notes.apply(lambda r: define(r.nfld_Expression), axis=1,result_type='expand')
kanjis.to_pickle('kanjis.pkl')

KeyboardInterrupt: 

In [178]:
notes.nfld_Expression.apply(len).max()

15